In [25]:
class Element:
    def __init__(self, tid, iutils, rutils):
        self.tid = tid           # Transaction id
        self.iutils = iutils     # Itemset utility
        self.rutils = rutils     # Remaining utility


In [26]:
class UtilityListEIHI:
    def __init__(self, item):
        """
        Constructor for UtilityListEIHI.

        :param item: The last item of the itemset represented by this utility list.
        """
        self.item = item
        self.sumIutilsD = 0  # The sum of iutil values of D
        self.sumRutilsD = 0  # The sum of rutil values of D
        self.sumIutilsDP = 0  # The sum of iutil values of D'
        self.sumRutilsDP = 0  # The sum of rutil values of D'
        self.elementsD = []  # The list of elements in this utility list for D
        self.elementsDP = []  # The list of elements in this utility list for D'

    def addElementD(self, element):
        """
        Method to add an element to this utility list and update the sums at the same time.

        :param element: The element to be added.
        """
        self.sumIutilsD += element.iutils
        self.sumRutilsD += element.rutils
        self.elementsD.append(element)

    def addElementDP(self, element):
        """
        Method to add an element to this utility list and update the sums at the same time.

        :param element: The element to be added.
        """
        self.sumIutilsDP += element.iutils
        self.sumRutilsDP += element.rutils
        self.elementsDP.append(element)

    def switchDPtoD(self):
        """
        Method to switch all elements from DP to D, and update the sums accordingly.
        """
        self.sumIutilsD += self.sumIutilsDP
        self.sumIutilsDP = 0
        self.sumRutilsD += self.sumRutilsDP
        self.sumRutilsDP = 0
        self.elementsD.extend(self.elementsDP)
        self.elementsDP.clear()


class AlgoEIHI:
    def __init__(self):
        self.maxMemory = 0
        self.startTimestamp = 0
        self.endTimestamp = 0
        self.huiCount = 0
        self.totalTimeForAllRuns = 0
        self.totalCandidateCountForAllRuns = 0
        self.transactionCount = 0
        self.candidateCount = 0
        self.mapItemToTWU = {}
        self.mapItemToRank = {}
        self.mapEUCS = {}
        self.debug = False
        self.mapItemToUtilityList = {}
        self.listOfUtilityLists = []
        self.totalDBUtility = 0
        self.minUtility = 0
        self.firstLine = 0
        self.itemsetBuffer = None
        self.BUFFERS_SIZE = 400
        self.singleItemsNodes = []

    class Pair:
        def __init__(self):
            self.item = 0
            self.utility = 0

        def __str__(self):
            return f"[{self.item},{self.utility}]"

    class Node:
        def __init__(self, item, utility=-1):
            self.item = item
            self.childs = []
            self.utility = utility

    def getRealHUICount(self):
        return self.getRealHUICountHelper(self.singleItemsNodes)

    def getRealHUICountHelper(self, listNodes):
        count = 0
        for node in listNodes:
            if node.utility >= self.minUtility:
                count += 1
            count += self.getRealHUICountHelper(node.childs)
        return count

    # def printHUIs(self):
    #     self.printHUIsHelper(self.singleItemsNodes, "")

    def printHUIsHelper(self, listNodes, prefix):
        for node in listNodes:
            itemset = f"{prefix} {node.item}"
            if len(itemset.strip().split()) > 1:  # Only print itemsets with 2 or more items
                if node.utility >= self.minUtility:
                    print(f"{itemset}  #UTIL: {node.utility}")
            if node.utility >= self.minUtility:
                print(f"{itemset}  #UTIL: {node.utility}")
            self.printHUIsHelper(node.childs, itemset)

    def writeHUIsToFile(self, output):
        seen = set()  # Để theo dõi các tập hợp đã được ghi
        with open(output, "w") as writer:
            self.writeHUIsToFileHelper(self.singleItemsNodes, "", writer, seen)

    def writeHUIsToFileHelper(self, listNodes, prefix, writer, seen):
        for node in listNodes:
            itemset = f"{prefix} {node.item}"
            if node.utility >= self.minUtility:
                if itemset not in seen:  # Kiểm tra trùng lặp
                    writer.write(f"{itemset}  #UTIL: {node.utility}\n")
                    seen.add(itemset)
            self.writeHUIsToFileHelper(node.childs, itemset, writer, seen)

    def printTrie(self):
        print("==== trie ====")
        self.printTrieHelper(self.singleItemsNodes, "")

    def printTrieHelper(self, listNodes, indent):
        for node in listNodes:
            itemset = f"{indent}{node.item}"
            print(f"{itemset}  ({node.utility})")
            self.printTrieHelper(node.childs, indent + "\t")

    def purgeTrie(self, listNodes):
        hasChildInHUI = False
        iterNodes = listNodes[:]
        for node in iterNodes:
            if node.utility >= self.minUtility:
                hasChildInHUI = True
            else:
                nodeHasChildInAHUI = self.purgeTrie(node.childs)
                if not nodeHasChildInAHUI:
                    listNodes.remove(node)
                else:
                    hasChildInHUI = True
        return hasChildInHUI

    def insertHUIinTrie(self, prefix, prefixLength, lastitem, utility):
        listNodes = self.singleItemsNodes
        currentNode = None

        for i in range(prefixLength):
            item = prefix[i]
            currentNode = self.binarySearchForItem(listNodes, item)
            if currentNode is None:
                currentNode = self.Node(item)
                listNodes.insert(self.middle, currentNode)
                listNodes = currentNode.childs
            else:
                listNodes = currentNode.childs

        currentNode = self.binarySearchForItem(listNodes, lastitem)
        if currentNode is None:
            currentNode = self.Node(lastitem, utility)
            listNodes.insert(self.middle, currentNode)
            self.huiCount += 1
        else:
            if currentNode.utility == -1:
                self.huiCount += 1
            currentNode.utility = utility

    def binarySearchForItem(self, listNodes, item):
        self.middle = 0
        first = 0
        last = len(listNodes) - 1

        while first <= last:
            self.middle = (first + last) // 2
            if self.compareItemsByRank(item, listNodes[self.middle].item) > 0:
                first = self.middle + 1
            elif self.compareItemsByRank(item, listNodes[self.middle].item) < 0:
                last = self.middle - 1
            else:
                return listNodes[self.middle]
        self.middle = first
        return None

    def runAlgorithm(self, input_file, minUtil, firstLine, lastLine):
        self.maxMemory = 0
        self.candidateCount = 0
        self.huiCount = 0
        self.itemsetBuffer = [0] * self.BUFFERS_SIZE
        self.firstLine = firstLine
        firstTime = self.mapEUCS is None

        if firstTime:
            self.mapEUCS = {}
            self.listOfUtilityLists = []
            self.mapItemToRank = {}
            self.mapItemToUtilityList = {}
            self.singleItemsNodes = []
            self.totalDBUtility = 0
        else:
            for ulist in self.listOfUtilityLists:
                ulist.switchDPtoD()

        self.startTimestamp = time.time()
        newItemsUtilityLists = []
        self.mapItemToTWU = {}

        with open(input_file, "r") as file:
            tid = 0
            for line in file:
                if tid >= lastLine:
                    break
                if tid >= firstLine:
                    if line.strip() == "" or line[0] in ("#", "%", "@"):
                        continue
                    split = line.split(":")
                    items = split[0].split(" ")
                    transactionUtility = int(split[1])
                    for item_str in items:
                        item = int(item_str)
                        twu = self.mapItemToTWU.get(item, 0)
                        twu += transactionUtility
                        self.mapItemToTWU[item] = twu
                        if item not in self.mapItemToUtilityList:
                            uList = UtilityListEIHI(item)
                            self.mapItemToUtilityList[item] = uList
                            newItemsUtilityLists.append(uList)
                    self.totalDBUtility += transactionUtility
                tid += 1

        self.minUtility = minUtil
        newItemsUtilityLists.sort(key=lambda ul: self.compareItems(ul.item, ul.item))
        for ul in newItemsUtilityLists:
            self.mapItemToRank[ul.item] = len(self.mapItemToRank) + 1
        self.listOfUtilityLists.extend(newItemsUtilityLists)

        # Second pass to construct utility lists of 1-itemsets with TWU >= minutil
        with open(input_file, "r") as file:
            tid = 0
            for line in file:
                if tid >= lastLine:
                    break
                if tid >= firstLine:
                    self.transactionCount += 1
                    split = line.split(":")
                    items = split[0].split(" ")
                    utilityValues = split[2].split(" ")
                    remainingUtility = 0
                    newTWU = 0
                    revisedTransaction = []
                    for i, item_str in enumerate(items):
                        pair = self.Pair()
                        pair.item = int(item_str)
                        pair.utility = int(utilityValues[i])
                        revisedTransaction.append(pair)
                        remainingUtility += pair.utility
                        newTWU += pair.utility
                    revisedTransaction.sort(key=lambda pair: self.compareItemsByRank(pair.item, pair.item))
                    for i, pair in enumerate(revisedTransaction):
                        remainingUtility -= pair.utility
                        utilityListOfItem = self.mapItemToUtilityList[pair.item]
                        element = Element(tid, pair.utility, remainingUtility)
                        utilityListOfItem.addElementDP(element)
                        mapFMAPItem = self.mapEUCS.get(pair.item, {})
                        for j in range(i + 1, len(revisedTransaction)):
                            pairAfter = revisedTransaction[j]
                            mapFMAPItem[pairAfter.item] = mapFMAPItem.get(pairAfter.item, 0) + newTWU
                        self.mapEUCS[pair.item] = mapFMAPItem
                tid += 1

        self.checkMemory()
        listULForRecursion = [ul for ul in self.listOfUtilityLists if ul.sumIutilsDP != 0]
        self.incFHM(self.itemsetBuffer, 0, None, listULForRecursion)
        self.checkMemory()
        self.endTimestamp = time.time()
        self.totalTimeForAllRuns += (self.endTimestamp - self.startTimestamp)
        self.totalCandidateCountForAllRuns += self.candidateCount

    def incFHM(self, prefix, prefixLength, pUL, ULs):
        for i in range(len(ULs)):
            X = ULs[i]
            utilityOfX = X.sumIutilsD + X            .sumIutilsDP
            if utilityOfX >= self.minUtility:
                if prefixLength > 0:  # Only insert itemsets with 2 or more items
                    self.insertHUIinTrie(prefix, prefixLength, X.item, utilityOfX)

            if X.sumIutilsDP + X.sumRutilsDP + X.sumIutilsD + X.sumRutilsD >= self.minUtility:
                exULs = []
                for j in range(i + 1, len(ULs)):
                    Y = ULs[j]

                    if Y.sumIutilsDP == 0:
                        continue

                    mapTWUF = self.mapEUCS.get(X.item)
                    if mapTWUF is not None:
                        twuF = mapTWUF.get(Y.item)
                        if twuF is None or twuF < self.minUtility:
                            continue

                    self.candidateCount += 1
                    temp = self.construct(pUL, X, Y)
                    if temp is not None:
                        exULs.append(temp)

                if prefixLength >= len(prefix):
                    prefix.append(X.item)
                else:
                    prefix[prefixLength] = X.item
                self.incFHM(prefix, prefixLength + 1, X, exULs)

    def compareItems(self, item1, item2):
        compare = self.mapItemToTWU[item1] - self.mapItemToTWU[item2]
        return (item1 - item2) if compare == 0 else compare

    def compareItemsByRank(self, item1, item2):
        compare = self.mapItemToRank[item1] - self.mapItemToRank[item2]
        return (item1 - item2) if compare == 0 else compare

    def construct(self, P, px, py):
        totalUtility = px.sumIutilsD + px.sumRutilsD + px.sumIutilsDP + px.sumRutilsDP
        pxyUL = UtilityListEIHI(py.item)

        for ex in reversed(px.elementsDP):
            ey = self.findElementWithTID(py.elementsDP, ex.tid)
            if ey is None:
                totalUtility -= (ex.iutils + ex.rutils)
                if totalUtility < self.minUtility:
                    return None
                continue

            if P is None:
                eXY = Element(ex.tid, ex.iutils + ey.iutils, ey.rutils)
                pxyUL.addElementDP(eXY)
            else:
                e = self.findElementWithTID(P.elementsDP, ex.tid)
                if e is not None:
                    eXY = Element(ex.tid, ex.iutils + ey.iutils - e.iutils, ey.rutils)
                    pxyUL.addElementDP(eXY)

        if not pxyUL.elementsDP:
            return None

        for ex in px.elementsD:
            ey = self.findElementWithTID(py.elementsD, ex.tid)
            if ey is None:
                totalUtility -= (ex.iutils + ex.rutils)
                if totalUtility < self.minUtility:
                    return None
                continue

            if P is None:
                eXY = Element(ex.tid, ex.iutils + ey.iutils, ey.rutils)
                pxyUL.addElementD(eXY)
            else:
                e = self.findElementWithTID(P.elementsD, ex.tid)
                if e is not None:
                    eXY = Element(ex.tid, ex.iutils + ey.iutils - e.iutils, ey.rutils)
                    pxyUL.addElementD(eXY)

        pxyUL.elementsDP.reverse()
        return pxyUL

    def findElementWithTID(self, list, tid):
        first = 0
        last = len(list) - 1

        while first <= last:
            middle = (first + last) // 2
            if list[middle].tid < tid:
                first = middle + 1
            elif list[middle].tid > tid:
                last = middle - 1
            else:
                return list[middle]
        return None

    def checkMemory(self):
        import psutil
        process = psutil.Process()
        currentMemory = process.memory_info().rss / 1024 / 1024
        if currentMemory > self.maxMemory:
            self.maxMemory = currentMemory

    def printStats(self):
        print("=============  EIHI ALGORITHM - STATS =============")
        print(f" Number of transactions processed: {self.transactionCount}")
        print(f" Execution time: {self.endTimestamp - self.startTimestamp:.2f} ms")
        print(f" Memory usage: {self.maxMemory:.2f} MB")
        print(f" New High-utility itemsets found: {self.huiCount}")
        print(f" Total high-utility itemsets count: {self.getRealHUICount()}")
        print(f" Candidate count: {self.candidateCount}")
        print(f" Min utility threshold: {self.minUtility}")
        print("===================================================")
        print(f" TOTAL DB Utility: {self.totalDBUtility}")
        print(f" TOTAL CANDIDATES FOR ALL RUNS: {self.totalCandidateCountForAllRuns}")
        print(f" TOTAL TIME FOR ALL RUNS: {self.totalTimeForAllRuns:.2f} ms")
        print("===================================================")



In [29]:
import os
import urllib.parse
import time

class MainTestEIHI:
    @staticmethod
    def main():
        # Initialize the algorithm
        algo = AlgoEIHI()

        # Set the minimum utility threshold
        min_utility = 30
        

        # 1) Apply the algorithm on a first file containing transactions
        print("1) Run the algorithm on the first file")

        input1 = MainTestEIHI.fileToPath("DB_UtilityIncremental1.txt")
        algo.runAlgorithm(input1, min_utility, 0, float('inf'))
        algo.printStats()

        # Print the number of HUIs found until now to the console
        realHUICount = algo.getRealHUICount()
        print(f"NUMBER OF HUI FOUND: {realHUICount}")

        # # PRINT THE HUIs FOUND
        # algo.printHUIs()

        # Write all the HUIs found until now to a file at any time with the following code
        output = "outputEIHI.txt"
        algo.writeHUIsToFile(output)

    @staticmethod
    def fileToPath(filename):
        # Sử dụng thư mục hiện tại
        current_dir = os.getcwd()  # Thay thế os.path.dirname(__file__) bằng os.getcwd()
        filepath = os.path.join(current_dir, filename)
        return urllib.parse.unquote(filepath)


if __name__ == "__main__":
    MainTestEIHI.main()


1) Run the algorithm on the first file
=============  EIHI ALGORITHM - STATS =============
 Number of transactions processed: 5
 Execution time: 0.00 ms
 Memory usage: 2485.61 MB
 New High-utility itemsets found: 8
 Total high-utility itemsets count: 8
 Candidate count: 37
 Min utility threshold: 30
 TOTAL DB Utility: 96
 TOTAL CANDIDATES FOR ALL RUNS: 37
 TOTAL TIME FOR ALL RUNS: 0.00 ms
NUMBER OF HUI FOUND: 8
